In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib import pyplot as plt
import numpy as np

from cassL import camb_interface as ci
from cassL import user_interface as ui
from cassL import utils

import camb
from camb import model, initialpower

import copy as cp

In [2]:
import importlib.util
import sys

spec = importlib.util.spec_from_file_location(
    "andreap", "../emulator/andreap.py")
andreap = importlib.util.module_from_spec(spec)
sys.modules["andreap"] = andreap
spec.loader.exec_module(andreap)

In [3]:
# Axe the 0.01 value since Ariel has not tested that one in the shared results
powernu3 = ci.load_benchmark(
    "../../benchmarks/power_nu3/Aletheia_powernu3_zorig_nu",
    ["0.0006", "0.0021", "0.0064", "0.0100"] 
)

spectra = ci.boltzmann_battery(ci.OMNUH2_FLOATS, skips_omega=[],
                               skips_model=[8], k_points=1000,
                               skips_snapshot=[])

np.save("Lukas_spectra.npy", spectra)

In [4]:
spectra = np.load("Lukas_spectra.npy", allow_pickle=True).item()

In [ ]:
Andrea_swarm = []
Ariel_swarm = []
k_axes = []

omnuh2_index = 0
for key in spectra:
    omnuh2 = key
    print(key)
 
    for modeli in range(len(spectra[key])):
        #print("model", modeli)
        if modeli == 8:
            continue
            
        model = ci.cosm.iloc[modeli]
        model = ci.specify_neutrino_mass(model, omnuh2, 1)
        
        z_array = ci.parse_redshifts(modeli)
        
        for snapi in range(len(spectra[key][modeli])):
            #print("snap", snapi)
            z = z_array[snapi]
            
            Lukas_spec = spectra[key][modeli][snapi]
            Ariel_spec = powernu3[ci.OMNUH2_STRS[omnuh2_index]][modeli][snapi]

            andrea_interpolator = andreap.cassL_to_andrea_cosmology(
                model, hybrid=False)
            
            if omnuh2_index == 0: # add the massless-neutrino case to our errors just once
                samples = np.zeros((2, 2, 1000))
                samples[0][0] = Ariel_spec["k"]
                samples[0][1] = Ariel_spec["P_no"]
                samples[1][0] = Lukas_spec["k"]
                samples[1][1] = Lukas_spec["P_no"]
                base_k, interpd_P = utils.homogenize_k_axes(samples)
            
                andrea_no_Pk = andrea_interpolator['mzero'].P(z, base_k)
                Andrea_errors = (andrea_no_Pk - interpd_P[1]) / andrea_no_Pk
                Andrea_swarm.append(Andrea_errors)
                Ariel_errors = (interpd_P[0] - interpd_P[1]) / interpd_P[0]
                Ariel_swarm.append(Ariel_errors)
                k_axes.append(base_k)
            
            samples = np.zeros((2, 2, 1000))
            samples[0][0] = Ariel_spec["k"]
            samples[0][1] = Ariel_spec["P_nu"]
            samples[1][0] = Lukas_spec["k"]
            samples[1][1] = Lukas_spec["P_nu"]
            base_k, interpd_P = utils.homogenize_k_axes(samples)
            
            
            
            andrea_Pk = andrea_interpolator['mnu'].P(z, base_k)
            Andrea_errors = (andrea_Pk - interpd_P[1]) / andrea_no_Pk
            Andrea_swarm.append(Andrea_errors)
            Ariel_errors = (interpd_P[0] - interpd_P[1]) / interpd_P[0]
            Ariel_swarm.append(Ariel_errors)
            k_axes.append(base_k)
            
    omnuh2_index += 1

0.0006356
[3.661418e+02 3.701280e+02 3.741575e+02 3.782307e+02 3.823483e+02
 3.865105e+02 3.907180e+02 3.949711e+02 3.992704e+02 4.036165e+02
 4.080097e+02 4.124506e+02 4.169397e+02 4.214776e+02 4.260647e+02
 4.307017e+02 4.353890e+02 4.401271e+02 4.449167e+02 4.497583e+02
 4.546523e+02 4.595995e+02 4.646003e+02 4.696554e+02 4.747653e+02
 4.799306e+02 4.851518e+02 4.904297e+02 4.957648e+02 5.011578e+02
 5.066092e+02 5.121197e+02 5.176899e+02 5.233204e+02 5.290120e+02
 5.347652e+02 5.405807e+02 5.464591e+02 5.524012e+02 5.584076e+02
 5.644790e+02 5.706162e+02 5.768197e+02 5.830903e+02 5.894288e+02
 5.958359e+02 6.023122e+02 6.088585e+02 6.154756e+02 6.221642e+02
 6.289251e+02 6.357590e+02 6.426667e+02 6.496491e+02 6.567067e+02
 6.638406e+02 6.710515e+02 6.783402e+02 6.857075e+02 6.931542e+02
 7.006813e+02 7.082895e+02 7.159798e+02 7.237528e+02 7.316096e+02
 7.395510e+02 7.475780e+02 7.556913e+02 7.638919e+02 7.721807e+02
 7.805587e+02 7.890268e+02 7.975860e+02 8.062371e+02 8.149811e+02


In [ ]:
np.save("Andrea_swarm.npy", Andrea_swarm)
np.save("Ariel_swarm.npy", Ariel_swarm)

In [ ]:
for i in range(len(Ariel_swarm)):
    plt.plot(k_axes[i], 100 * Ariel_swarm[i], color="blue", alpha=0.2)

    plt.xscale('log')
plt.ylabel("% error between CL and Sánchez codes")
plt.xlabel("Inverse scale $k$")
plt.title("Error Curves Using Sánchez's Spectra")

In [ ]:
for i in range(len(Andrea_swarm)):
    plt.plot(k_axes[i], 100 * Andrea_swarm[i], color="blue", alpha=0.05)

    plt.xscale('log')
plt.ylabel("% error between CL and Pezzotta codes")
#plt.ylim((-0.2, 0.15))
plt.xlabel("Inverse scale $k$")
plt.title("Error Curves Using Pezzotta's Spectra")